In [4]:
import pandas as pd
import numpy as np
import re
path = "C:/Users/user/Documents/tcc/dados/"
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
from IPython.display import display_html
from itertools import chain,cycle
import time

def display_side_by_side(*args,titles=cycle([''])):
    html_str=''
    for df,title in zip(args, chain(titles,cycle(['</br>'])) ):
        html_str+='<th style="text-align:center"><td style="vertical-align:top">'
        html_str+=f'<h2 style="text-align: center;">{title}</h2>'
        html_str+=df.to_html().replace('table','table style="display:inline"')
        html_str+='</td></th>'
    display_html(html_str,raw=True)

def duration_time_parser(string):
    string = str(string)
    string = re.sub('s', '',re.sub(r'h|m', ':', str(string)))
    lista = re.split(':', string, 3)
    if string.lower() == "nan":
        return None         
    elif len(lista) == 1:
        segundos = int(lista[-1])
    elif len(lista) == 2:
        segundos = (int(lista[-2]) * 60) + int(lista[-1])
    elif len(lista) == 3:
        segundos = (int(lista[-3]) * 3600) + (int(lista[-2]) * 60) + int(lista[-1])
    else:
        return None  
    time_format = time.strftime("%H:%M:%S", time.gmtime(segundos))
    return time_format

def duration_time_seconds(string):
    string = str(string)
    string = re.sub('s', '',re.sub(r'h|m', ':', str(string)))
    lista = re.split(':', string, 3)
    if string.lower() == "nan":
        return None         
    elif len(lista) == 1:
        segundos = int(lista[-1])
    elif len(lista) == 2:
        segundos = (int(lista[-2]) * 60) + int(lista[-1])
    elif len(lista) == 3:
        segundos = (int(lista[-3]) * 3600) + (int(lista[-2]) * 60) + int(lista[-1])
    else:
        return None   
    return segundos

In [26]:
df_base = pd.read_csv(f"{path}DF_STREAM_PT_NOME_IDS_wqtdemotes.csv", sep=";")
df_base = df_base.loc[(df_base.total_followers >= 30)] #Publico alvo(canais com pelo menos 60% da meta de afiliado ou mais)
display(df_base.head(5))
df_base = df_base[['broadcaster_id', 'total_followers', 'team_id', 'qtd_emotes']]
display(df_base.head(5))
df_base.to_csv(f'{path}base_independentes/base.csv', mode='w', sep=";", header=True, index=False)

,broadcaster_id,broadcaster_login,broadcaster_name,total_followers,team_id,qtd_emotes
0,23631109,xruk,xRUK,87,0,0
1,42642296,muuhpro,MuuhPro,229804,0,32
2,62916113,focasp,FocaSP,34720,0,21
3,29361707,zarbaofc,zarbaofc,151,0,0
4,62290505,xtojah,xtojah,8694,0,0


,broadcaster_id,total_followers,team_id,qtd_emotes
0,23631109,87,0,0
1,42642296,229804,0,32
2,62916113,34720,0,21
3,29361707,151,0,0
4,62290505,8694,0,0


In [10]:
df_tags = pd.read_csv(f"{path}canal_tags.csv", sep=";")
display(df_tags.head(5))
print('qtd registros:', len(df_tags))
print("qtd de tags unicas:", len(df_tags.localization_names.unique())) #há mais de 349 tags nessa base de dados
df_tags = df_tags.drop_duplicates() #tratar registros unicos
print('qtd registros unicos:', len(df_tags))
lista_tags = df_tags[["broadcaster_id", "localization_names"]].drop_duplicates()

lista_tags.to_csv(f'{path}base_independentes/user_tags.csv', mode='w', sep=";", header=True, index=False)

,tag_id,is_auto,localization_names,localization_descriptions,broadcaster_id
0,39ee8140-901a-4762-bfca-8260dea1310f,True,Português,Para transmissões que utilizam o idioma português.,42642296
1,39ee8140-901a-4762-bfca-8260dea1310f,True,Português,Para transmissões que utilizam o idioma português.,62916113
2,988c8c1b-b991-4b66-9a6d-b36f60363f6d,False,Brasil,Para transmissões e conteúdos com participação da comunidade brasileira,29361707
3,e1a43486-eee5-4f36-9ee6-76e72af00180,False,Jogada de ranking,Para transmissões focadas em competições em que o objetivo é atingir colocações em placares.,29361707
4,e659959d-392f-44c5-83a5-fb959cdbaccc,False,100%,Para transmissões focadas em completar jogos em 100%,29361707


qtd registros: 6121
qtd de tags unicas: 314
qtd registros unicos: 6121


In [27]:
df_tags = pd.read_csv(f"{path}canal_tags.csv", sep=";")
print('qtd registros:', len(df_tags))
print("qtd de tags unicas:", len(df_tags.localization_names.unique())) #há mais de 349 tags nessa base de dados
df_tags = df_tags.drop_duplicates() #tratar registros unicos
print('qtd registros unicos:', len(df_tags))
lista_tags = df_tags[["localization_names", "localization_descriptions"]].drop_duplicates()

lista_tags.to_csv(f'{path}base_independentes/tags.csv', mode='w', sep=";", header=True, index=False)

qtd registros: 6121
qtd de tags unicas: 314
qtd registros unicos: 6121


In [12]:
df_clips = pd.read_csv(f"{path}clips_2022-05-20_a_2022-08-20.csv", sep=";", encoding='iso-8859-1') #Os clips estão sem categoria (usar só max e média?)
df_clips = df_clips.sort_values(by=['view_count'], ascending=False)
display(df_clips.head(5))
df_clips = df_clips[['broadcaster_id', 'game_id', 'view_count', 'duration']].rename(columns={"game_id": "clip_game_id", 'view_count': 'clip_view_count', 'duration':'clip_duration'})
display(df_clips.head(5))
df_clips.to_csv(f'{path}base_independentes/clips_games.csv', mode='w', sep=";", header=True, index=False)

,id,url,embed_url,broadcaster_id,broadcaster_name,creator_id,creator_name,video_id,game_id,language,title,view_count,created_at,thumbnail_url,duration,pagination
3012,StylishSullenHabaneroArgieB8-5uuntQg5LCQ38J6n,https://clips.twitch.tv/StylishSullenHabaneroArgieB8-5uuntQg5LCQ38J6n,https://clips.twitch.tv/embed?clip=StylishSullenHabaneroArgieB8-5uuntQg5LCQ38J6n,75947497,s2alinefaria,254193053,kschorro_do_mangue,NaN,116747788.0,pt-br,entreguei o baiano,91909,2022-08-09T22:45:39Z,https://clips-media-assets2.twitch.tv/qba2Lebrp7UIOSwkN_Cepw/40046882777-offset-1574-preview-480x272.jpg,30.0,NaN
3013,AnnoyingJollyIcecreamLeeroyJenkins-LsPg5AZlaEEDLZ2a,https://clips.twitch.tv/AnnoyingJollyIcecreamLeeroyJenkins-LsPg5AZlaEEDLZ2a,https://clips.twitch.tv/embed?clip=AnnoyingJollyIcecreamLeeroyJenkins-LsPg5AZlaEEDLZ2a,75947497,s2alinefaria,174990494,Daviizaoo,NaN,116747788.0,pt-br,veguinha,62932,2022-08-04T23:35:30Z,https://clips-media-assets2.twitch.tv/O3-N-S7D4OQGZWhBAy2dmg/40022187881-offset-894-preview-480x272.jpg,30.0,NaN
3014,BrainyAdorableWaffleTTours-TuE8GpNC4VWJTto4,https://clips.twitch.tv/BrainyAdorableWaffleTTours-TuE8GpNC4VWJTto4,https://clips.twitch.tv/embed?clip=BrainyAdorableWaffleTTours-TuE8GpNC4VWJTto4,75947497,s2alinefaria,799414038,bernardin444344,NaN,116747788.0,pt-br,ta krai,51280,2022-08-09T22:45:33Z,https://clips-media-assets2.twitch.tv/UD0wg07hPBqTQuFViU7aOA/AT-cm%7CUD0wg07hPBqTQuFViU7aOA-preview-480x272.jpg,59.9,NaN
3015,TransparentBraveEggBrokeBack-yx8jW_ONKIvTQZqL,https://clips.twitch.tv/TransparentBraveEggBrokeBack-yx8jW_ONKIvTQZqL,https://clips.twitch.tv/embed?clip=TransparentBraveEggBrokeBack-yx8jW_ONKIvTQZqL,75947497,s2alinefaria,109277671,bronnks7,NaN,116747788.0,pt-br,kkkk,33292,2022-08-09T23:10:04Z,https://clips-media-assets2.twitch.tv/Bzdf-csfI1vzjSiL00Gaiw/40046882777-offset-3040-preview-480x272.jpg,30.0,NaN
52182,ShinyPolishedLatteSeemsGood-YtYjXnOoZogrx81Y,https://clips.twitch.tv/ShinyPolishedLatteSeemsGood-YtYjXnOoZogrx81Y,https://clips.twitch.tv/embed?clip=ShinyPolishedLatteSeemsGood-YtYjXnOoZogrx81Y,122675528,Hirowdots,53235445,loogw,1.513269e+09,509658.0,pt-br,parabens,32897,2022-06-25T02:52:56Z,https://clips-media-assets2.twitch.tv/AT-cm%7Cnfk4emLWI15zqGaIK8fNWA-preview-480x272.jpg,42.9,6424.0


,broadcaster_id,clip_game_id,clip_view_count,clip_duration
3012,75947497,116747788.0,91909,30.0
3013,75947497,116747788.0,62932,30.0
3014,75947497,116747788.0,51280,59.9
3015,75947497,116747788.0,33292,30.0
52182,122675528,509658.0,32897,42.9


In [5]:
df_videos = pd.read_csv(f"{path}videos_with_chapters_novo.csv", sep=";", encoding='iso-8859-1') #pode ser que alguns registros tenham um titulo que prejudiquem o dataframe df_videos['view_count'] == 'pt' (a base de treinamento foi corrida manualmente, mas pode poder ocorrer em "produção")
print('qtd registros df_videos:', df_videos.shape)
df_videos['duration_time'] = df_videos.duration.apply(duration_time_parser)
df_videos['duration_seconds'] = df_videos.duration.apply(duration_time_seconds)
display(df_videos.head(5))
df_videos = df_videos[['user_id', 'game_id', 'view_count', 'duration_seconds']].rename(columns={'user_id':'broadcaster_id',
                                                                                                "game_id": "video_game_id", 
                                                                                                'view_count': 'video_view_count',
                                                                                                'duration_seconds':'video_duration_seconds'})
display(df_videos.head(5))
df_videos.to_csv(f'{path}base_independentes/videos_games.csv', mode='w', sep=";", header=True, index=False)

qtd registros df_videos: (47432, 20)


,id,stream_id,user_id,user_login,user_name,title,description,created_at,published_at,url,thumbnail_url,viewable,view_count,language,type,duration,muted_segments,game_id,game_name,game_duration,duration_time,duration_seconds
0,1566966657,1,42642296,muuhpro,MuuhPro,Hazel Sky - Parte 1/InÃ­cio,NaN,2022-08-20 00:45:25+00:00,2022-08-20T00:45:25Z,https://www.twitch.tv/videos/1566966657,https://static-cdn.jtvnw.net/cf_vods/d2nvs31859zcd8/8f9e922425c77771c284_muuhpro_33396531370_5866245686/thumb/custom-0f22b465-a861-4969-96c6-7c3fc906d1c2-%{width}x%{height}.jpeg,public,67.0,pt,VideoType.HIGHLIGHT,24m47s,NaN,516587.0,Hazel Sky,NaN,00:24:47,1487.0
1,1566715632,1,62916113,focasp,FocaSP,Destaque: !potatosrp | Seu Adalberto em serviÃ§o <3,NaN,2022-08-19 19:56:50+00:00,2022-08-19T19:56:50Z,https://www.twitch.tv/videos/1566715632,https://static-cdn.jtvnw.net/cf_vods/d1m7jfoe9zdc1j/7bdce5238ad5e779a2ad_focasp_93063639266_2432970080//thumb/thumb1566715632-%{width}x%{height}.jpg,public,2.0,pt,VideoType.HIGHLIGHT,4h11m20s,NaN,32982.0,Grand Theft Auto V,NaN,04:11:20,15080.0
2,1566107414,40091482665,62916113,focasp,FocaSP,!potatosrp | Seu Adalberto em serviÃ§o <3,NaN,2022-08-19 02:25:36+00:00,2022-08-19T02:25:36Z,https://www.twitch.tv/videos/1566107414,https://static-cdn.jtvnw.net/cf_vods/d1m7jfoe9zdc1j/3aee2950b0bd938bd7a9_focasp_40091482665_1660875932//thumb/thumb0-%{width}x%{height}.jpg,public,3090.0,pt,VideoType.ARCHIVE,4h11m20s,NaN,32982.0,Grand Theft Auto V,NaN,04:11:20,15080.0
3,1560046282,1,62916113,focasp,FocaSP,Destaque: !potatosrp | Seu Adalberto em serviÃ§o <3,NaN,2022-08-12 19:42:09+00:00,2022-08-12T19:42:09Z,https://www.twitch.tv/videos/1560046282,https://static-cdn.jtvnw.net/cf_vods/d1m7jfoe9zdc1j/96293e4eff7e9414bfb3_focasp_66270880770_9823912542//thumb/thumb1560046282-%{width}x%{height}.jpg,public,3.0,pt,VideoType.HIGHLIGHT,3h18m40s,NaN,32982.0,Grand Theft Auto V,NaN,03:18:40,11920.0
4,1559385187,40057427737,62916113,focasp,FocaSP,!potatosrp | Seu Adalberto em serviÃ§o <3,NaN,2022-08-12 01:19:58+00:00,2022-08-12T01:19:58Z,https://www.twitch.tv/videos/1559385187,https://static-cdn.jtvnw.net/cf_vods/d1m7jfoe9zdc1j/9a87c18cee3a1add5dd0_focasp_40057427737_1660267194//thumb/thumb0-%{width}x%{height}.jpg,public,2366.0,pt,VideoType.ARCHIVE,3h18m40s,NaN,32982.0,Grand Theft Auto V,NaN,03:18:40,11920.0


,broadcaster_id,video_game_id,video_view_count,video_duration_seconds
0,42642296,516587.0,67.0,1487.0
1,62916113,32982.0,2.0,15080.0
2,62916113,32982.0,3090.0,15080.0
3,62916113,32982.0,3.0,11920.0
4,62916113,32982.0,2366.0,11920.0
